# Templating

In a hipotetical website were we have:

- Home Page (with all post listed)
- Post Page (with the post details)

we should setup a script capable to  collect data from each page according to the templates.

here we adapt the our old script to this website, all the paths need to be updated

In [16]:
import requests
from lxml import html

# instead of print, collect all the post links and url of the paged you scraped

start_url = "https://www.immobiliare.it/Torino/case_in_vendita-Torino.html?criterio=rilevanza"

# here we create some empty lists that will collect all your data
urls_to_scrape = []
urls_scraped = []
urls_posts = []

# in this list are stored all the paths needed to parse on the homepage

post_paths = []

# //*[@id="listing-container"]/li[1]/div[2]/div[1]/p/a/@href
# //*[@id="listing-container"]/li[3]/div[2]/div[1]/p/a/@href
# //*[@id="listing-container"]/li[31]/div[2]/div[1]/p/a/@href

# create the paths
for i in range (1,32):
    post_paths.append('//*[@id="listing-container"]/li['+str(i)+']/div[2]/div[1]/p/a/@href')
    
# this is the path of the link of next page
nextpage_path = '//*[@id="listing-pagination"]/ul[3]/li[1]/a/@href'

# a counter, we will use it to stop the script after a certain point
counter = 0 
run_n_times = 4

# our first definition of main
def main():
    
    # if the list of links to scrape is not empty
    global counter 
    if urls_to_scrape:
        
        # take the fist element of the list  
        url = urls_to_scrape[0]
        # open the url
        r = requests.get(url)
        # transform the pure text in HTML 
        tree = html.fromstring(r.text)
        # for all the path in paths 
        for path in post_paths:
            # extract the url 
            u = tree.xpath(path)
            # if not empty 
            if u:
                # take the first item in the list
                u = u[0]
                # add the url to the posts url list
                urls_posts.append(u)
                # print the address
                print(u)
            
        
        # get the next page (it comes in '/2/' format)
        nextpage = tree.xpath(nextpage_path)
        # here we join the start_url 'http://www.macroumors.com' with the '/2/'
        urls_to_scrape.append(start_url+nextpage[0])
        # than we add the new url to the we scraped
        urls_scraped.append(urls_to_scrape[0])
        # now we take out the first element in the list of the one to scrape (it was the homepage)
        urls_to_scrape.pop(0)
        # print the last url you got
        print(urls_scraped)
        # add a item into the counter 
        counter = counter + 1
        
        # here we assure that after a certain numeber i stop
        if counter < run_n_times :
            # if is less of the run_n_times run all main again 
            main()
              
    else:
        # if the list is empty, add the start url
        urls_to_scrape.append(start_url)
        # than run main 
        main()
    

# here we initialize the script
main()


https://www.immobiliare.it/68144075-Vendita-Trilocale-corso-Luigi-Cibrario-Chieri.html
https://www.immobiliare.it/68143243-Vendita-Villa-via-Montamise-37-San-Sebastiano-da-Po.html
https://www.immobiliare.it/68144403-Vendita-Trilocale-corso-Lecce-52-Torino.html
https://www.immobiliare.it/68139855-Vendita-Quadrilocale-via-Barletta-125-Torino.html
https://www.immobiliare.it/68139845-Vendita-Appartamento-via-Pianezza-68-Torino.html
https://www.immobiliare.it/68139927-Vendita-Appartamento-corso-Francia-20-Torino.html
https://www.immobiliare.it/68144557-Vendita-Villa-via-dei-Mille-9BIS-Moncalieri.html
https://www.immobiliare.it/68139837-Vendita-Trilocale-via-Lanzo-71-Torino.html
https://www.immobiliare.it/68139861-Vendita-Trilocale-via-San-Donato-78-Bis-Torino.html
https://www.immobiliare.it/68144311-Vendita-Trilocale-via-Santa-Lucia-68-Mathi.html
https://www.immobiliare.it/68139887-Vendita-Quadrilocale-via-Sansovino-125-Torino.html
https://www.immobiliare.it/68139841-Vendita-Quadrilocale-co

In [17]:
# our first definition of SinglePost class
class SinglePost():
    def __init__(self,url):
        self.url = url
        self.paths = { 
                      'title':'//*[@id="sticky-contact-bottom"]/div[1]/h1/text()',
                      'price':'//*[@id="sticky-contact-bottom"]/div[1]/div[1]/div[1]/ul[1]/li[1]/span/text()',
                      'rooms':'//*[@id="sticky-contact-bottom"]/div[1]/div[1]/div[1]/ul[2]/li[1]/div[1]/span/text()',
                      'bathrooms':'//*[@id="sticky-contact-bottom"]/div[1]/div[1]/div[1]/ul[2]/li[2]/div[1]/span/text()',
                      'area':'//*[@id="sticky-contact-bottom"]/div[1]/div[1]/div[1]/ul[2]/li[3]/div[1]/span/text()'
                     }
        
        # here we have some vars waiting to be assigned
        self.r = None
        self.tree = None
        self.data = {}
    
    # a function that is just waiting to be 
    def openUrl(self):
        self.r = requests.get(self.url)
        self.tree = html.fromstring(self.r.text)
    
    # check if the data if ok avoiding errors
    def controlData(self,data):
        f = None
        if data:
            f = data[0].replace('€','').replace('\xa0','').replace('.','')
        else:
            f = ''
        return f
    
    # extract the data from the xpath
    def getData(self):
        for key, value in self.paths.items():
            d = { key : self.controlData(self.tree.xpath(value)) }
            self.data.update(d)
    
    # just show our data 
    def show(self):
        print(self.data)
    
    # all things together
    def run(self):
        self.openUrl()
        self.getData()
        self.show()
            

In [22]:
import time

sleep_time = 2 # seconds

# our second definition of main
def main():
    
    # if the list of links to scrape is not empty 
    if urls_to_scrape:
        
        # take the fist element of the list  
        url = urls_to_scrape[0]
        # open the url
        r = requests.get(url)
        # transform the pure text in HTML 
        tree = html.fromstring(r.text)
        # for all the path in paths 
        time.sleep(sleep_time)
        for path in post_paths:
            # extract the url 
            u = tree.xpath(path)
            # if not empty 
            if u:
                # take the first item in the list
                u = u[0]
                # add the url to the posts url list
                urls_posts.append(u)                
                
                # now we introduce our Class
                s = SinglePost(u)
                s.run()
                      
                    
        # get the next page (it comes in '/2/' format)
        nextpage = tree.xpath(nextpage_path)
        # here we join the start_url 'http://www.macroumors.com' with the '/2/'
        urls_to_scrape.append(start_url+nextpage[0])
        # than we add the new url to the we scraped
        urls_scraped.append(urls_to_scrape[0])
        # now we take out the first element in the list of the one to scrape (it was the homepage)
        urls_to_scrape.pop(0)
        # print the last url you got
        print(urls_scraped)
        # add a item into the counter 
        counter = counter + 1
        
        # here we assure that after a certain numeber i stop
        if counter < run_n_times :
            # if is less of the run_n_times run all main again 
            main()
              
    else:
        # if the list is empty, add the start url
        urls_to_scrape.append(start_url)
        # than run main 
        main()
    

# here we initialize the script
main()
            

{'title': 'Trilocale corso Luigi Cibrario, Chieri', 'price': ' 135000', 'rooms': '3', 'bathrooms': '1', 'area': '74'}
{'title': 'Villa via Montamisè 37, San Sebastiano da Po', 'price': ' 328000', 'rooms': '4', 'bathrooms': '2', 'area': '244'}
{'title': 'Trilocale corso Lecce 52, Torino', 'price': ' 169000', 'rooms': '3', 'bathrooms': '1', 'area': '85'}
{'title': 'Quadrilocale via Barletta, 125, Torino', 'price': ' 160000', 'rooms': '4', 'bathrooms': '1', 'area': '91'}
{'title': 'Appartamento via Pianezza, 68, Torino', 'price': ' 175000', 'rooms': '5', 'bathrooms': '2', 'area': '140'}
{'title': 'Appartamento corso Francia, 20, Torino', 'price': ' 650000', 'rooms': '5', 'bathrooms': '2', 'area': '150'}
{'title': 'Villa via dei Mille 9BIS, Moncalieri', 'price': ' 329000', 'rooms': '5', 'bathrooms': '1', 'area': '243'}
{'title': 'Trilocale via Lanzo, 71, Torino', 'price': ' 57000', 'rooms': '3', 'bathrooms': '1', 'area': '63'}
{'title': 'Trilocale via San Donato, 78 Bis, Torino', 'price': 